In [1]:
!pip install flask-ngrok
!pip install transformers pillow
!pip install flask-cors
!pip install pyngrok
!pip install Pillow
!pip install torch
!pip install torchvision
!pip install transformers
!pip install sentencepiece
!pip install accelerate
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.6 MB/s eta 0:00:00


In [2]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from transformers import AutoModel, AutoTokenizer
from PIL import Image
import torch
from pyngrok import ngrok
from flask_cors import CORS

In [3]:
model = AutoModel.from_pretrained('openbmb/MiniCPM-Llama3-V-2_5-int4', trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-Llama3-V-2_5-int4', trust_remote_code=True)
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

configuration_minicpm.py:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-Llama3-V-2_5-int4:
- configuration_minicpm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_minicpmv.py:   0%|          | 0.00/25.0k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

MiniCPMV(
  (llm): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(128256, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      

In [4]:
# Khởi tạo Flask app và ngrok
app = Flask(__name__)

In [11]:
# Khởi tạo CORS cho ứng dụng Flask
CORS(app)

In [22]:
# Hàm inference với câu hỏi
def model_inference(image, question, msgs):
  res = model.chat(
      image=image,
      msgs=msgs,
      tokenizer=tokenizer,
      sampling=True, # if sampling=False, beam_search will be used by default
      temperature=0.7,
      # system_prompt='' # pass system_prompt if needed
  )
  return res

In [47]:
@app.route('/analyze-book-cover-v4', methods=['POST'])
def analyze_book_cover_v4():
    # Lấy ảnh từ request
    if 'file' not in request.files:
        return jsonify({"error": "No file provided"}), 400

    file = request.files['file']
    image = Image.open(file.stream)

    # Đặt câu hỏi cho từng thông tin cần lấy
    title_question = "What is the title of the book?"
    author_question = "Who is the author of the book?"
    publisher_question = "Who is the publisher of the book?"

    title_msgs = [{'role': 'user', 'content': title_question}]
    author_msgs = [{'role': 'user', 'content': author_question}]
    publisher_msgs = [{'role': 'user', 'content': publisher_question}]

    # Chạy qua model và lấy kết quả
    title = model_inference(image, title_question, title_msgs)
    author = model_inference(image, author_question, author_msgs)
    publisher = model_inference(image, publisher_question, publisher_msgs)

    return jsonify({
        "title": title,
        "author": author,
        "publisher": publisher
    })

In [48]:
!ngrok authtoken 2oqd79lxFI4SKnDYejYVU670TX7_KyFCNd9RzPh3eTmrpS4d

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Mở server Flask
app_port = 5000
ngrok_tunnel = ngrok.connect(app_port)
print(f"Ngrok tunnel created: {ngrok_tunnel.public_url}")

app.run(port=app_port)

Ngrok tunnel created: https://871a-34-168-22-90.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 10:33:57] "POST /analyze-book-cover-v4 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 10:34:16] "POST /analyze-book-cover-v4 HTTP/1.1" 200 -
